## Segmenting interlayers

Welcome! 
In this tutorial we train a neural network to segment interlayers on seismic cube `P_cube`

In [1]:
import sys
import os

In [2]:
sys.path.append('../repos/seismiqb/')

In [3]:
from seismiqb import SeismicCropBatch, SeismicCubeset
from seismiqb.batchflow import FilesIndex, D, L, F, Pipeline, B, V
from seismiqb.batchflow.models.tf import UNet
import numpy as np
from tqdm import tqdm_notebook, tqdm
import matplotlib.pyplot as plt
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_VISIBLE_DEVICES=2


* Indexing seismic cubes: setting up `SeismicCubeset`

In [4]:
DATA_DIR = '/notebooks/data/cubes/CUBE_3/'

In [5]:
qbix = FilesIndex(path=os.path.join(DATA_DIR, 'P_cube.sgy'), no_ext=True)
qbset = SeismicCubeset(qbix)

In [6]:
qbset.load_geometries()

Working with CUBE_3/P_cube.sgy: 100%|██████████| 7113463/7113463 [01:23<00:00, 85277.79it/s]


* load horizon-labels

In [7]:
pref = os.path.join(DATA_DIR, 'HORIZONTS')
horizons_paths = [os.path.join(pref, p) for p in ['prb_B_anon.txt',
                                                  'prb_T_ALL_anon.txt',
                                                  'prb_G_anon.txt',
                                                  'prb_M_anon.txt']]

In [8]:
qbset.load_point_clouds(paths={qbset.indices[0]: horizons_paths})

In [9]:
qbset.load_labels()

* make train/test samplers

In [10]:
qbset.load_samplers()

In [54]:
qbset.train_sample = qbset.sampler.truncate(expr=lambda x: x[:, 1:],
                                            high=[1, 0.3, 1]).sample
qbset.test_sample = qbset.sampler.truncate(expr=lambda x: x[:, 1:],
                                           low=[0, 0.65, 0]).sample